In [29]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler

import matplotlib.pyplot as plt

In [30]:
# single layer lstm
class LSTM(nn.Module):
    def __init__(self, n_hidden=64):
        super(LSTM, self).__init__()
        self.n_hidden = n_hidden
        
        # layers
        self.lstm = nn.LSTMCell(1, self.n_hidden)
        self.linear = nn.Linear(self.n_hidden, 1)
        
    def forward(self, x, future=0):
        outputs = []
        
        # hidden states and cell states
        h_t0 = torch.zeros(x.size(0), self.n_hidden, dtype=torch.float32)
        c_t0 = torch.zeros(x.size(0), self.n_hidden, dtype=torch.float32)
        
        for input_t in x.split(1, dim=1):
            h_t1, c_t1 = self.lstm(input_t, (h_t0, c_t0))
            output = self.linear(h_t1)
            outputs += [output]
            
        for i in range(future): # if we should predict the future
            h_t1, c_t1 = self.lstm(input_t, (h_t0, c_t0))
            output = self.linear(h_t1)
            outputs += [output]
            
        outputs = torch.cat(outputs, dim=1)
        return outputs

In [31]:
inputs = np.array([[200],[400],[600],[800],[1000],[1200],[1400],[1600],[1800],[2000]], dtype=np.float32)
target = np.array([[80],[160],[240],[320],[400],[480.0], [560.0], [640.0], [720.0], [800.0]], dtype=np.float32)

sc1 = MinMaxScaler(feature_range=(0,1))
sc2 = MinMaxScaler(feature_range=(0,1))

sc1.fit(inputs)
sc2.fit(target)

inputs = sc1.transform(inputs)
target = sc2.transform(target)

train_input = torch.from_numpy(inputs)   # [7,1], [0:7]
train_target = torch.from_numpy(target)  # [7,1], [0:7]
test_input = torch.from_numpy(inputs)    # [3,1], [7:]
test_target = torch.from_numpy(target)   # [3,1], [7:]

inputs, target

(array([[0.        ],
        [0.11111111],
        [0.22222224],
        [0.33333334],
        [0.44444448],
        [0.5555556 ],
        [0.6666667 ],
        [0.7777778 ],
        [0.8888889 ],
        [1.        ]], dtype=float32),
 array([[0.        ],
        [0.11111111],
        [0.22222224],
        [0.33333334],
        [0.44444448],
        [0.5555556 ],
        [0.6666667 ],
        [0.7777778 ],
        [0.8888889 ],
        [1.        ]], dtype=float32))

In [26]:
model = LSTM()
criterion = nn.MSELoss()
optimizer = optim.LBFGS(model.parameters(), lr=0.01)

n_steps = 500

# training loop
for i in range(n_steps):
    def closure():
        optimizer.zero_grad()               # zero the gradient
        out = model(train_input)            # forward pass
        loss = criterion(out, train_target) # calculate the loss
        loss.backward()                     # backward pass        
        return loss
    
    optimizer.step(closure)                 # update parameters
    
    with torch.no_grad():                   # testing
        future = 1
        pred = model(test_input, future=future)
        loss = criterion(pred[:, :-1], test_target)
        y = pred.detach().numpy()

        if i % 50 == 0:
            print(f"Loss in training: {loss.item()}")
            print(f"Test loss: {loss.item()}")

            
def predict(x):
    x_scaled = (x - 200) / (2000 - 200)
    y_actual = x * 0.4
    input_t = torch.tensor([[x_scaled]])
    pred = model(input_t)
    pred_val = pred.detach().numpy()
    y_scaled = pred_val.item() * (800 - 80) + 80
    print("x_scaled", x_scaled)
    print("pred", y_scaled)
    print("y_actual", y_actual)
    
predict(876)


# x_flatten = sc.inverse_transform(test_input).flatten()
# y_flatten = sc.inverse_transform(test_target).flatten()
# pred_flatten = sc.inverse_transform(y[:, :-1]).flatten()

# plt.figure(figsize=(12,6))
# plt.xlabel('X')
# plt.ylabel('Y')


# plt.plot(x_flatten, y_flatten, 'r')
# plt.plot(x_flatten, pred_flatten, 'b--')
# plt.show()

# print(x_flatten)
# print(y_flatten)
# print(pred_flatten)

# print(test_input, test_target)

Loss in training: 0.28811925649642944
Test loss: 0.28811925649642944
Loss in training: 9.702205261419294e-07
Test loss: 9.702205261419294e-07
Loss in training: 9.329788781542447e-07
Test loss: 9.329788781542447e-07
Loss in training: 9.095459745367407e-07
Test loss: 9.095459745367407e-07
Loss in training: 8.943133025240968e-07
Test loss: 8.943133025240968e-07
Loss in training: 8.841403769110912e-07
Test loss: 8.841403769110912e-07
Loss in training: 8.772332193984766e-07
Test loss: 8.772332193984766e-07
Loss in training: 8.725072007109702e-07
Test loss: 8.725072007109702e-07
Loss in training: 8.691378639014147e-07
Test loss: 8.691378639014147e-07
Loss in training: 8.667512929605437e-07
Test loss: 8.667512929605437e-07
x_scaled 0.37555555555555553
pred 349.9644660949707
y_actual 350.40000000000003


In [27]:
# save the model
PATH = 'single_lstm.pt'
torch.save(model.state_dict(), PATH)

In [28]:
# load and export the model to onnx

input_names = ["actual_input"]
output_names = ["output"]

trained_model = LSTM()
trained_model.load_state_dict(torch.load('single_lstm.pt'))
trained_model.eval()
dummy_input = torch.tensor([[0.1]])
torch.onnx.export(trained_model, 
                  dummy_input, 
                  'single_lstm-new.onnx', 
                  verbose=True,
                 input_names=input_names,
                 output_names=output_names,
                 export_params=True)